In [34]:
import numpy as np, math, pandas as pd, pprint

def getDataFile():
    filename = "example.xls"
    return pd.read_excel(filename)

def improveDataFileForAccuracy(datafile):
    values = datafile.values
    m = np.size(values,0) 
    n = np.size(values,1)
    p = []
    for i in range (0,m):
        for j in range(0,n):
            if isinstance(values[i,j], basestring) or np.isnan(values[i,j]):
                p.append(i)
                break
    datafile = np.delete(values, p, axis = 0)
    datafile = datafile[:,51:61]
    return datafile

def getDataFrame():
    datafile = getDataFile()
    datafile = improveDataFileForAccuracy(datafile)
    return datafile

def getDataFromTable():
    Z = getDataFrame() 
    Z = Z.astype("float64")
    return Z

def getMeanOfColumn(df,index):
    rowscount = np.size(df, 0)
    sum = 0
    for i in range(0,rowscount):
        sum = df[i, index] + sum
    return sum/rowscount

def getDispersionOfColumn(df,index):
    mean = getMeanOfColumn(df,index)   
    sumOfSq = 0
    rowscount = np.size(df,0)
    for i in range(0, rowscount):
        sumOfSq = math.pow(df[i,index] - mean,2) + sumOfSq
    return sumOfSq/rowscount

def getStandardMatrix(df):
    rowscount = np.size(df,0)
    columnscount = np.size(df,1)
    standardMatrix = np.zeros((rowscount,columnscount))
    for i in range(0,columnscount):
        for j in range(0,rowscount):
            standardMatrix[j,i] = (df[j,i] - getMeanOfColumn(df,i))/math.sqrt(getDispersionOfColumn(df,i))
    return standardMatrix

def getCovMatrix(df):
    rowscount = np.size(df,0)
    columnscount = np.size(df,1)
    covMatrix = np.zeros((columnscount,columnscount))
    for i in range(0,columnscount):
        for j in range(0,columnscount):
            value = 0
            for k in range(0, rowscount):
                value = value + (df[k,i] - getMeanOfColumn(df,i))*(df[k,j] - getMeanOfColumn(df,j))
            covMatrix[i,j] = value/rowscount
    return covMatrix

def getCorrMatrix(df):
    rowscount = np.size(df,0)
    columnscount = np.size(df,1)
    corrMatrix = np.zeros((columnscount,columnscount))
    standardMatrix = getStandardMatrix(df)
    for i in range(0, columnscount):
        for j in range(0,columnscount):
            value = 0
            for k in range(0,rowscount):
                value = standardMatrix[k,i]*standardMatrix[k,j] + value
            corrMatrix[i,j] = value/rowscount
    return corrMatrix

def changeInDescendingOrder(ownNumbers,yacobMatrix):
    for k in range(0,len(ownNumbers)):
        for i in range(0,len(ownNumbers) - 1):
            if ownNumbers[i] < ownNumbers[i+1]:
                a = ownNumbers[i]
                ownNumbers[i] = ownNumbers[i+1]
                ownNumbers[i+1] = a
                yacobMatrix[:,[i,i + 1]] = yacobMatrix[:,[i + 1, i]]
    
np.set_printoptions(precision=3,suppress=True)
Z = getDataFromTable()
StandZ =  getStandardMatrix(Z)
CovZ = getCovMatrix(StandZ)
lambdas = (np.linalg.eig(CovZ))[0]
C = (np.linalg.eig(CovZ))[1]
changeInDescendingOrder(lambdas,C)
y = np.zeros((33,10))
for i in range(10):
    y = np.dot(Z,np.transpose(C))
# print "Проекции объектов на главные компоненты:\n",y
# np.dot(np.transpose(yacobMatrix),yacobMatrix)
# D1 = np.dot(np.dot(np.transpose(C[:,0]),CovZ),C[:,0])
# print D1
# D2 = np.dot(np.dot(np.transpose(C[:,1]),CovZ),C[:,1])
# print D2
# D3 = np.dot(np.dot(np.transpose(C[:,2]),CovZ),C[:,2])
# print D3
# D4 = np.dot(np.dot(np.transpose(C[:,3]),CovZ),C[:,3])
# print D4

print C

def getInfoAboutDispersions():
    sum1 = 0
    sum2 = 0
    for i in range(10):
        sum1 += getDispersionOfColumn(Z,i)
        sum2 += getDispersionOfColumn(y,i)
    print "Сумма выборочных дисперсий исходных признаков: ", sum1, "\nСумма выборочной дисперсии вычисленных проекций: ",sum2

# print np.cov(getStandardMatrix(y))
def getParameterI():
    sum1 = 0
    p = 0
    I = 0
    sum2 = 0
    for j in range(10):
        sum2 += getDispersionOfColumn(y,j)
    while (I < 0.95):
        sum1 += getDispersionOfColumn(Z,p)
        I = sum1/sum2
        p+=1
    print "значение I(p'): ", I
    print "Значение p': ", p + 1

for i in range(8):
    print "y", i+1 ," = ", 
    for j in range(10):
        if j != 9:
            print round(C[j,i],3), "*x",j + 1,"+",
        else:
            print round(C[j,i],3), "*x",j + 1
# getInfoAboutDispersions()
# getParameterI()

[[-0.351 -0.028 -0.551 -0.202  0.028 -0.265 -0.298 -0.372  0.481  0.051]
 [-0.354  0.024 -0.492 -0.307  0.255  0.139  0.164  0.54  -0.366 -0.033]
 [-0.2    0.143  0.509 -0.75  -0.192 -0.063 -0.008  0.162  0.225  0.015]
 [-0.358  0.326  0.007  0.058 -0.047  0.52   0.543 -0.429  0.089 -0.051]
 [-0.354  0.353  0.198  0.043  0.098 -0.353 -0.27  -0.339 -0.623  0.002]
 [-0.286  0.354  0.243  0.39   0.422  0.135 -0.312  0.367  0.392  0.018]
 [ 0.284  0.423 -0.081 -0.018  0.279 -0.569  0.534  0.052  0.158  0.143]
 [ 0.366 -0.047  0.05  -0.364  0.711  0.3   -0.163 -0.321 -0.056 -0.008]
 [ 0.293  0.473 -0.204 -0.077 -0.204  0.066 -0.197  0.048  0.019 -0.745]
 [ 0.276  0.466 -0.217 -0.078 -0.289  0.275 -0.263  0.049 -0.078  0.646]]
y 1  =  -0.351 *x 1 + -0.354 *x 2 + -0.2 *x 3 + -0.358 *x 4 + -0.354 *x 5 + -0.286 *x 6 + 0.284 *x 7 + 0.366 *x 8 + 0.293 *x 9 + 0.276 *x 10
y 2  =  -0.028 *x 1 + 0.024 *x 2 + 0.143 *x 3 + 0.326 *x 4 + 0.353 *x 5 + 0.354 *x 6 + 0.423 *x 7 + -0.047 *x 8 + 0.473 *x 9 + 0